# Extracción Transformación y Carga (ETL)

### Fuente de datos:
+ [DataSet de Google Maps !!!](https://drive.google.com/drive/folders/1Wf7YkxA0aHI3GpoHc9Nh8_scf5BbD4DA?usp=drive_link):  Google Drive Henry.

+ [DataSet de Yelp !!!](https://drive.google.com/drive/folders/1TI-SsMnZsNP6t930olEEWbBQdo_yuIZF?usp=drive_link):  Google Drive Henry..
<br/>
- - -

Importando Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime, timedelta
import random
from scipy.stats import norm

import ast
import json
import os                       # Libreria para manejo de arhivos y directorios del sistema
import glob                     # crear listas de archivos a partir de búsquedas con comodines en un directorio
import pyarrow as pa
import jsonlines                # Permite procesar un registro por vez
import pyarrow.parquet as pq    # Permite leer archivos en formato parquet
import pickle5 as pickle        # Permite leer archivos con formatp pkl, para archivos version 5

- - -
### Funciones generales

Funcion que permite buscar archivos en un directorio segun un filtro deseado

In [2]:
# Recibe el directorio a trabajar y el patron o filtro que se desea mostrar.

def listar_archivos_segun_patron_con_directorio(directorio, patron):
    return glob.glob(directorio + patron)

- - -
+ DataSet de Yelp !!!
- - -

`Datasets: checkin`

Tipo de archivo JSon

Procedimieto que permite separar el dataset checkin en 1 archivos tipo JSon.<br>
El datasets checkin tiene aproximadamente: 131.930 registros, por tamaño del archivo debe ser separado para que pueda ser subido al repositorio.<br>
131930 // 2 aproximadamente 65965<br>
Se abre el archivo JSon y se salva tipo parquet con la misma cantidad de registros.

In [5]:
archivo_checkin_json = "../Datasets/Datasets_Yelp/checkin/checkin.json"
cantidad_registros = 65965                                                      # Cantidad de registros por archivos
with jsonlines.open(archivo_checkin_json) as reader:
    lista_datos = list(reader)                                                  # Lee todos los registros y crea una lista
    total_registros = len(lista_datos)                                          # Se tiene el total de registros, esto para separar en varios archivos.
    for ind in range(0, total_registros, cantidad_registros):
        iformacion_registros = lista_datos[ind:ind+cantidad_registros]          # Se extrae grupos de cantidad_registros registros
        df = pd.DataFrame(iformacion_registros)                                 # Se crea un dataframe con el grupo de registros extraidos
        table = pa.Table.from_pandas(df)                                        # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
        nombre_archivo = "checkin_" + str(ind//cantidad_registros) + ".parquet" # Se arma el nombre del archivo
        ruta_archivo = os.path.join('../Datasets/Datasets_Yelp/checkin/', nombre_archivo)    # La ruta donde se guardara el archivo
        pq.write_table(table, ruta_archivo)                                     # Se escribe en el directorio

Se cargan los datasets que contienen grupos de registros ya separados. 

In [28]:
directorio = "../Datasets/Datasets_Yelp/checkin/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "checkin_*.parquet")
df_list = []                                    # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                      # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_user = pq.read_table(source=archivo)  # Se crea una tabla con cada archivo
    df = tabla_user.to_pandas()                 # Se transforma la tabla en un dataframe
    df_list.append(df)                          # Se adicionan a los dataframe
df_checkin_concatenado = pd.concat(df_list)     # Se concatenan todos los DataFrames en uno solo
df_checkin_concatenado = df_checkin_concatenado.reset_index(drop=True)    # Se resetea el indice a uno secuencial y consecutivo

In [29]:
df_checkin_concatenado.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [30]:
df_checkin_concatenado.shape

(131930, 2)

Diccionario del dataset checkin

| # | Column | Dtype | RealType |
| --------- | --------- | --------- | --------- |
| 0 | business_id | object | string |
| 3 | date | float64 | list date YYYY-MM-DD |

Para su manipulacion se reduce el dataset a 5 años de registros y se ajustan los campos:<br> 
. date al formato: YYYY-MM-DD<br>

In [27]:
print('Datos iniciales desde: ', df_checkin_concatenado['date'].min(), 'hasta: ',df_checkin_concatenado['date'].max())
# df_tip_concatenado['date'] = pd.to_datetime(df_tip_concatenado['date'])
# df_tip_2018_to_2022 = df_tip_concatenado[df_tip_concatenado['date'].dt.year.between(2018, 2022)]
# df_tip_2018_to_2022 = df_tip_2018_to_2022.reset_index(drop=True)
# df_tip_2018_to_2022['date'] = pd.to_datetime(df_tip_2018_to_2022['date']).dt.strftime('%Y-%m-%d')
# print('Datos finales desde: ', df_tip_2018_to_2022['date'].min(), 'hasta: ',df_tip_2018_to_2022['date'].max())

Datos iniciales desde:  2009-12-30 02:53:27, 2010-02-11 18:43:45, 2010-04-25 03:57:16, 2010-08-07 01:59:21, 2010-09-27 23:11:18, 2010-10-01 23:16:35, 2010-10-14 03:05:41, 2010-10-15 14:41:50, 2010-10-16 13:02:26, 2010-10-23 23:30:18, 2010-11-14 16:44:16, 2010-11-26 18:14:29, 2010-11-27 16:12:46, 2010-12-01 12:30:33, 2010-12-20 21:11:15, 2011-01-21 23:19:57, 2011-02-02 16:30:10, 2011-02-05 03:00:15, 2011-02-05 03:52:29, 2011-03-13 17:52:54, 2011-03-15 22:22:34, 2011-04-08 16:21:27, 2011-04-09 17:04:31, 2011-05-14 17:17:47, 2011-05-14 17:18:35, 2011-05-14 17:18:37, 2011-05-15 03:02:25, 2011-06-05 22:58:18, 2011-06-14 23:52:02, 2011-07-24 17:03:19, 2011-07-28 15:25:57, 2011-07-30 14:33:13, 2011-07-30 14:42:04, 2011-08-06 05:16:21, 2011-08-18 05:05:33, 2011-08-29 00:40:44, 2011-10-13 20:40:52, 2011-10-29 10:41:31, 2011-11-02 16:11:44, 2011-11-02 16:11:58, 2011-11-03 16:30:57, 2011-11-06 05:11:24, 2011-11-06 15:02:36, 2011-11-07 18:33:14, 2011-11-12 19:46:30, 2011-11-13 14:25:42, 2011-11-13

In [33]:
table = pa.Table.from_pandas(df_checkin_concatenado)                              # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
nombre_archivo = "checkin-reducido.parquet"                                       # Se arma el nombre del archivo
ruta_archivo = os.path.join('../Datasets/Datasets_Yelp/checkin/', nombre_archivo) # La ruta donde se guardara el archivo
pq.write_table(table, ruta_archivo)                                               # Se graba en el directorio

Debido a limitaciones del repositorio de GitHob, donde solo permite archivos de tamaño maximo de 100MB, se procede a generar archivos que no excedan dicho tamaño.<br>
. Cantidad total de registros:  131930<br>
. Cantidad de registros aproximado por archivos: 32982<br>
. Cantidad de archivos: 4

In [12]:
archivo_checkin_parquet = "../Datasets/Datasets_Yelp/checkin/checkin-reducido.parquet"  # Ruta y nombre del Archivo original
datos_checkin_parquet = pq.read_table(archivo_checkin_parquet)                          # Se lee y se carga el archivo como una tabla
cantidad_registros = 32982                                                              # Cantidad de registros por archivos separados
for ind in range(0, datos_checkin_parquet.num_rows, cantidad_registros):                # Un ciclo para grupo de registros
    info_cantidad_registros = datos_checkin_parquet.slice(ind, cantidad_registros)      # Se extrae grupos de cantidad_registros registros
    nombre_archivo = "checkin-reducido_" + str(ind // cantidad_registros) + ".parquet"  # Se arma el nombre del archivo
    ruta_archivo = os.path.join('../Datasets/Datasets_Yelp/checkin/', nombre_archivo)   # La ruta donde se guardara el archivo
    pq.write_table(info_cantidad_registros, ruta_archivo)                               # Se graba en el directorio

Cargando los archivos parquet reducidos como un Dataframe

In [4]:
directorio = "../Datasets/Datasets_Yelp/checkin/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "checkin-reducido_*.parquet")
df_list = []                                        # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                          # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_checkin = pq.read_table(source=archivo)   # Se crea una tabla con cada archivo
    df = tabla_checkin.to_pandas()                  # Se transforma la tabla en un dataframe
    df_list.append(df)                              # Se adicionan a los dataframe
df_checkin_reducido_concatenado = pd.concat(df_list)# Se concatenan todos los DataFrames en uno solo
df_checkin_reducido_concatenado = df_checkin_reducido_concatenado.reset_index(drop=True)  # Se resetea el indice a uno secuencial y consecutivo

In [9]:
df_checkin_reducido_concatenado.head()

,business_id,date
0,kPU91CF4Lq2WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


- - -

In [11]:
# Se le quita el caracter '-' del campo business_id
df_checkin_reducido_concatenado['business_id'] = df_checkin_reducido_concatenado['business_id'].str.replace('-','',regex=True)

In [17]:
# df_checkin_reducido_concatenado = df_checkin_reducido_concatenado.explode('date')

# df_checkin_reducido_concatenado.date.str.split(pat=',',expand=True)

In [18]:
print(df_checkin_reducido_concatenado['date'])

0         2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020...
1         2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011...
2                  2013-06-14 23:29:17, 2014-08-13 23:20:22
3         2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012...
4         2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014...
                                ...                        
131925    2013-03-23 16:22:47, 2013-04-07 02:03:12, 2013...
131926                                  2021-06-12 01:16:12
131927    2011-05-24 01:35:13, 2012-01-01 23:44:33, 2012...
131928             2016-12-03 23:33:26, 2018-12-02 19:08:45
131929                                  2015-01-06 17:51:53
Name: date, Length: 131930, dtype: object


In [10]:
# Guarda un dataframe en formato parquet
archivo_checkin_parquet = "../Datasets/Datasets_Yelp/checkin/checkin-reducido.parquet"
df_checkin_reducido_concatenado.to_parquet(archivo_checkin_parquet)

- - -
`Datasets: tip`

Tipo de archivo JSon

Procedimieto que permite separar el dataset tip en 1 archivos tipo parquet.<br>
El datasets tip tiene aproximadamente: 908.915 registros.<br>
Se abre el archivo JSon y se salva tipo parquet con la misma cantidad de registros.

In [ ]:
archivo_tip_json = "../Datsets/Datasets_Yelp/tip/review.json"

with jsonlines.open(archivo_tip_json) as reader:
    lista_datos = list(reader)                                                  # Lee todos los registros y crea una lista
    total_registros = len(lista_datos)                                          # Se tiene el total de registros, esto para separar en varios archivos.

    for ind in range(0, total_registros, total_registros):
        iformacion_registros = lista_datos[ind:ind+total_registros]             # Se extrae grupos de cantidad_registros registros
        df = pd.DataFrame(iformacion_registros)                                 # Se crea un dataframe con el grupo de registros extraidos
        table = pa.Table.from_pandas(df)                                        # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
        nombre_archivo = "tip_" + str(ind//total_registros) + ".parquet"        # Se arma el nombre del archivo
        ruta_archivo = os.path.join('../Datsets/Datasets_Yelp/tip/', nombre_archivo)    # La ruta donde se guardara el archivo
        pq.write_table(table, ruta_archivo)

Se cargan los datasets que contienen grupos de registros ya separados. 

In [12]:
directorio = "../Datasets/Datasets_Yelp/tip/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "tip_*.parquet")
df_list = []                                    # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                      # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_user = pq.read_table(source=archivo)  # Se crea una tabla con cada archivo
    df = tabla_user.to_pandas()                 # Se transforma la tabla en un dataframe
    df_list.append(df)                          # Se adicionan a los dataframe
df_tip_concatenado = pd.concat(df_list)        # Se concatenan todos los DataFrames en uno solo
df_tip_concatenado = df_tip_concatenado.reset_index(drop=True)    # Se resetea el indice a uno secuencial y consecutivo

Diccionario del dataset tip

| # | Column | Dtype | RealType |
| --------- | --------- | --------- | --------- |
| 0 | user_id | object | string |
| 1 | business_id | object | string |
| 2 | text | object | string |
| 3 | date | float64 | date YYYY-MM-DD |
| 4 | compliment_count | int64 | int64 |

Para su manipulacion se reduce el dataset a 5 años de registros y se ajustan los campos:<br> 
. date al formato: YYYY-MM-DD<br>

In [17]:
print('Datos iniciales desde: ', df_tip_concatenado['date'].min(), 'hasta: ',df_tip_concatenado['date'].max())
df_tip_concatenado['date'] = pd.to_datetime(df_tip_concatenado['date'])
df_tip_2018_to_2022 = df_tip_concatenado[df_tip_concatenado['date'].dt.year.between(2018, 2022)]
df_tip_2018_to_2022 = df_tip_2018_to_2022.reset_index(drop=True)
df_tip_2018_to_2022['date'] = pd.to_datetime(df_tip_2018_to_2022['date']).dt.strftime('%Y-%m-%d')
print('Datos finales desde: ', df_tip_2018_to_2022['date'].min(), 'hasta: ',df_tip_2018_to_2022['date'].max())

Datos iniciales desde:  2009-04-16 13:11:49 hasta:  2022-01-19 20:38:55
Datos finales desde:  2018-01-01 hasta:  2022-01-19


Se crea un archivo en el directorio tip con el dataset reducido en cantidad de registros para 5 años

In [18]:
table = pa.Table.from_pandas(df_tip_2018_to_2022)                              # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
nombre_archivo = "tip-reducido.parquet"                                        # Se arma el nombre del archivo
ruta_archivo = os.path.join('../Datasets/Datasets_Yelp/tip/', nombre_archivo)  # La ruta donde se guardara el archivo
pq.write_table(table, ruta_archivo)                                            # Se graba en el directorio

- - -
2 años de registros

In [4]:
directorio = "../Datasets/Datasets_Yelp/tip/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "tip-reducido*.parquet")
df_list = []                                        # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                          # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_review = pq.read_table(source=archivo)    # Se crea una tabla con cada archivo
    df = tabla_review.to_pandas()                   # Se transforma la tabla en un dataframe
    df_list.append(df)                              # Se adicionan a los dataframe
df_tip_reducido_concatenado = pd.concat(df_list) # Se concatenan todos los DataFrames en uno solo
df_tip_reducido_concatenado = df_tip_reducido_concatenado.reset_index(drop=True)  # Se resetea el indice a uno secuencial y consecutivo

In [9]:
print('Datos iniciales desde: ', df_tip_reducido_concatenado['date'].min(), 'hasta: ',df_tip_reducido_concatenado['date'].max())
df_tip_reducido_concatenado['date'] = pd.to_datetime(df_tip_reducido_concatenado['date'])
df_tip_2018_to_2020 = df_tip_reducido_concatenado[df_tip_reducido_concatenado['date'].dt.year.between(2018, 2020)]
df_tip_2018_to_2020 = df_tip_2018_to_2020.reset_index(drop=True)
df_tip_2018_to_2020['date'] = pd.to_datetime(df_tip_2018_to_2020['date']).dt.strftime('%Y-%m-%d')
print('Datos finales desde: ', df_tip_2018_to_2020['date'].min(), 'hasta: ',df_tip_2018_to_2020['date'].max())

Datos iniciales desde:  2018-01-01 00:00:00 hasta:  2022-01-19 00:00:00
Datos finales desde:  2018-01-01 hasta:  2020-12-31


In [6]:
df_tip_reducido_concatenado.head()

,user_id,business_id,text,date,compliment_count
0,1viszljzsa8W7VnYPuWosw,Zg83olSjsdXAhE5EEy5IcQ,Order crispy and they bake it up good.,2018-04-01,0
1,YnlCpuaBa3qWBp4te8pGmA,XIKYdKWq72zUYsq8NBxcCQ,The honey glazed salmon is amazing!,2018-01-14,0
2,XJDLaoN1PerKw2woiKeepA,t0zwddmbOGQOADrAxEPHQQ,Come by for some Jelly Bean sours this weekend!,2018-03-31,0
3,9M_nk0kA3CTJt6UoxRAK_g,oJf3IZloLf_PDeEeZW5nDg,Meatball sandwich,2018-03-27,0
4,0LY6l9-Z2IrVtnfj6OX06w,Zmwm6d872C8kWJr4b6UGfg,Great tasty food,2018-04-01,0


- - -

Debido a limitaciones del repositorio de GitHob, donde solo permite archivos de tamaño maximo de 100MB.
En el caso del dataset tip, no es necesario separar en varios archivos ya que su tamaño se ajusta.<br>
. Cantidad total de registros:  193033<br>
. Cantidad de archivos: 1

Cargando los archivos parquet reducidos como un Dataframe

In [21]:
directorio = "../Datasets/Datasets_Yelp/tip/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "tip-reducido*.parquet")
df_list = []                                        # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                          # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_review = pq.read_table(source=archivo)    # Se crea una tabla con cada archivo
    df = tabla_review.to_pandas()                   # Se transforma la tabla en un dataframe
    df_list.append(df)                              # Se adicionan a los dataframe
df_tip_reducido_concatenado = pd.concat(df_list) # Se concatenan todos los DataFrames en uno solo
df_tip_reducido_concatenado = df_tip_reducido_concatenado.reset_index(drop=True)  # Se resetea el indice a uno secuencial y consecutivo

In [22]:
df_tip_reducido_concatenado.head()

,user_id,business_id,text,date,compliment_count
0,1viszljzsa8W7VnYPuWosw,Zg83olSjsdXAhE5EEy5IcQ,Order crispy and they bake it up good.,2018-04-01,0
1,YnlCpuaBa3qWBp4te8pGmA,XIKYdKWq72zUYsq8NBxcCQ,The honey glazed salmon is amazing!,2018-01-14,0
2,XJDLaoN1PerKw2woiKeepA,t0zwddmbOGQOADrAxEPHQQ,Come by for some Jelly Bean sours this weekend!,2018-03-31,0
3,9M_nk0kA3CTJt6UoxRAK_g,oJf3IZloLf_PDeEeZW5nDg,Meatball sandwich,2018-03-27,0
4,0LY6l9-Z2IrVtnfj6OX06w,Zmwm6d872C8kWJr4b6UGfg,Great tasty food,2018-04-01,0


- - -
`Datasets: review`

Tipo de archivo JSon

Procedimieto que permite separar el dataset review en 13 archivos tipo parquet con aproximadamente 500.000 registros cada archivo.<br>
El datasets reviews tiene aproximadamente: 6.990.280 registros.<br>

El siguiente procedimiento lee el datasets original JSon, aproximadamente de 5Gb y lo separa en archivos tipo parquet.

In [ ]:
archivo_review_json = "../Datsets/Datasets_Yelp/review/review.json"             # Ruta y nombre del Archivo original
cantidad_registros = 500000                                                     # Cantidad de registros por archivos
with jsonlines.open(archivo_review_json) as reader:                             # Se lee el archivo original JSon
    lista_datos = list(reader)                                                  # Lee todos los registros y crea una lista
    total_registros = len(lista_datos)                                          # Se tiene el total de registros, esto para separar en varios archivos.
    for ind in range(0, total_registros, cantidad_registros):                   # Un ciclo por cada grupo de registors
        iformacion_registros = lista_datos[ind:ind+cantidad_registros]          # Se extrae grupos de cantidad_registros registros
        df = pd.DataFrame(iformacion_registros)                                 # Se crea un dataframe con el grupo de registros extraidos
        table = pa.Table.from_pandas(df)                                        # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
        nombre_archivo = "review_" + str(ind//cantidad_registros) + ".parquet"  # Se arma el nombre del archivo
        ruta_archivo = os.path.join('../Datsets/Datasets_Yelp/review/', nombre_archivo) # La ruta donde se guardara el archivo
        pq.write_table(table, ruta_archivo)                                     # Se graba en el directorio

Se cargan los datasets que contienen grupos de registros ya separados. 

In [4]:
directorio = "../Datasets/Datasets_Yelp/review/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "review_*.parquet")
df_list = []                                    # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                      # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_user = pq.read_table(source=archivo)  # Se crea una tabla con cada archivo
    df = tabla_user.to_pandas()                 # Se transforma la tabla en un dataframe
    df_list.append(df)                          # Se adicionan a los dataframe
df_review_concatenado = pd.concat(df_list)        # Se concatenan todos los DataFrames en uno solo
df_review_concatenado = df_review_concatenado.reset_index(drop=True)    # Se resetea el indice a uno secuencial y consecutivo

Diccionario del dataset review

| # | Column | Dtype | RealType |
| --------- | --------- | --------- | --------- |
| 0 | review_id | object | string |
| 1 | user_id | object | string |
| 2 | business_id | object | string |
| 3 | stars | float64 | int64 |
| 4 | useful | int64 | int64 |
| 5 | funny | int64 | int64 |
| 6 | cool | int64 | int64 |
| 7 | text | object | string |
| 8 | date | object | date YYYY-MM-DD |

Para su manipulacion se reduce el dataset a 5 años de registros y se ajustan los campos:<br> 
. date al formato: YYYY-MM-DD<br>
. starts a tipo int64

In [25]:
print('Datos iniciales desde: ', df_review_concatenado['date'].min(), 'hasta: ',df_review_concatenado['date'].max())
df_review_concatenado['date'] = pd.to_datetime(df_review_concatenado['date'])
df_review_2018_to_2022 = df_review_concatenado[df_review_concatenado['date'].dt.year.between(2018, 2022)]
df_review_2018_to_2022 = df_review_2018_to_2022.reset_index(drop=True)
df_review_2018_to_2022['date'] = pd.to_datetime(df_review_2018_to_2022['date']).dt.strftime('%Y-%m-%d')
df_review_2018_to_2022['stars'] = df_review_2018_to_2022['stars'].astype('int64')
print('Datos finales desde: ', df_review_2018_to_2022['date'].min(), 'hasta: ',df_review_2018_to_2022['date'].max())

Datos iniciales desde:  2005-02-16 03:23:22 hasta:  2022-01-19 19:48:45
Datos finales desde:  2018-01-01 hasta:  2022-01-19


Se crea un archivo en el directorio review con el dataset reducido en cantidad de registros para 5 años

In [28]:
table = pa.Table.from_pandas(df_review_2018_to_2022)                              # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
nombre_archivo = "review-reducido.parquet"                                        # Se arma el nombre del archivo
ruta_archivo = os.path.join('../Datasets/Datasets_Yelp/review/', nombre_archivo)  # La ruta donde se guardara el archivo
pq.write_table(table, ruta_archivo)                                               # Se graba en el directorio

Debido a limitaciones del repositorio de GitHob, donde solo permite archivos de tamaño maximo de 100MB, se procede a generar archivos que no excedan dicho tamaño.<br>
. Cantidad total de registros:  2742210<br>
. Cantidad de registros aproximado por archivos: 100000<br>
. Cantidad de archivos: 27

Procedimiento para crear archivos separados de grupos de registros donde el tamaño de cada archivo no exceda de 50Mb

In [17]:
archivo_review_parquet = "../Datasets/Datasets_Yelp/review/review-reducido.parquet" # Ruta y nombre del Archivo original
datos_review_parquet = pq.read_table(archivo_review_parquet)                        # Se lee y se carga el archivo como una tabla
cantidad_registros = 100000                                                         # Cantidad de registros por archivos separados
for ind in range(0, datos_review_parquet.num_rows, cantidad_registros):             # Un ciclo para grupo de registros
    info_cantidad_registros = datos_review_parquet.slice(ind, cantidad_registros)   # Se extrae grupos de cantidad_registros registros
    nombre_archivo = "review-reducido_" + str(ind // cantidad_registros) + ".parquet"   # Se arma el nombre del archivo
    ruta_archivo = os.path.join('../Datasets/Datasets_Yelp/review/', nombre_archivo)    # La ruta donde se guardara el archivo
    pq.write_table(info_cantidad_registros, ruta_archivo)                           # Se graba en el directorio

Cargando los archivos parquet reducidos como un Dataframe

In [3]:
directorio = "../Datasets/Datasets_Yelp/review/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "review-reducido_*.parquet")
df_list = []                                        # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                          # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_review = pq.read_table(source=archivo)    # Se crea una tabla con cada archivo
    df = tabla_review.to_pandas()                   # Se transforma la tabla en un dataframe
    df_list.append(df)                              # Se adicionan a los dataframe
df_review_reducido_concatenado = pd.concat(df_list) # Se concatenan todos los DataFrames en uno solo
df_review_reducido_concatenado = df_review_reducido_concatenado.reset_index(drop=True)  # Se resetea el indice a uno secuencial y consecutivo

In [4]:
df_review_reducido_concatenado.shape

(2742210, 9)

In [23]:
# Se le quita la columna funny
df_review_reducido_concatenado =  df_review_reducido_concatenado.drop('funny', axis=1)

In [24]:
df_review_reducido_concatenado.head(4).T

,0,1,2,3
review_id,KU_O5udG6zpxOg-VcAEodg,lUUhg8ltDsUZ9h0xnwY4Dg,JBWZmBy69VMggxj3eYn17Q,E9AB7V4z8xrt2uPF7T55FQ
user_id,mh_-eMZ6K5RLWhZyISBhwA,RreNy--tOmXMl1en0wiBOg,aFa96pz67TwOFu4Weq5Agg,iYY5Ii1LGpZCpXFkHlMefw
business_id,XQfwVwDr-v0ZS3_CbbE5Xw,cPepkJeRMtHapc_b2Oe_dw,kq5Ghhh14r-eCxlVmlyd8w,Zx7n8mdt8OzLRXVzolXNhQ
stars,3,4,5,5
useful,0,1,0,0
cool,0,1,0,0
text,"If you decide to eat here, just be aware it is...",I was really between 3 and 4 stars for this on...,My boyfriend and I tried this deli for the fir...,Amazing biscuits and (fill in the blank). Grea...
date,2018-07-07,2018-07-17,2018-08-23,2018-04-27


In [26]:
# Creando un archivo en formato csv

archivo_review_csv = "../Datasets/Datasets_Yelp/review/review.csv"
df_review_reducido_concatenado.to_csv(archivo_review_csv, index=False)

In [27]:
archivo_review_csv = "../Datasets/Datasets_Yelp/review/review.csv"
df = pd.read_csv(archivo_review_csv)

In [28]:
df.head().T

,0,1,2,3,4
review_id,KU_O5udG6zpxOg-VcAEodg,lUUhg8ltDsUZ9h0xnwY4Dg,JBWZmBy69VMggxj3eYn17Q,E9AB7V4z8xrt2uPF7T55FQ,A4n4YaE-owOVgTQcrVqHUw
user_id,mh_-eMZ6K5RLWhZyISBhwA,RreNy--tOmXMl1en0wiBOg,aFa96pz67TwOFu4Weq5Agg,iYY5Ii1LGpZCpXFkHlMefw,S7bjj-L07JuRr-tpX1UZLw
business_id,XQfwVwDr-v0ZS3_CbbE5Xw,cPepkJeRMtHapc_b2Oe_dw,kq5Ghhh14r-eCxlVmlyd8w,Zx7n8mdt8OzLRXVzolXNhQ,I6L0Zxi5Ww0zEWSAVgngeQ
stars,3,4,5,5,4
useful,0,1,0,0,0
cool,0,1,0,0,0
text,"If you decide to eat here, just be aware it is...",I was really between 3 and 4 stars for this on...,My boyfriend and I tried this deli for the fir...,Amazing biscuits and (fill in the blank). Grea...,The cafe was extremely cute. We came at 8am an...
date,2018-07-07,2018-07-17,2018-08-23,2018-04-27,2018-07-07


Cargando el archivo parquet como una table pyarrow

In [15]:
archivo_review_parquet = "../Datasets/Datasets_Yelp/review/review-reducido.parquet"
datos_tabla_parquet = pq.read_table(archivo_review_parquet)

In [16]:
print("Tipo de dato: ",type(datos_tabla_parquet))
print("Tipos de datos: ",datos_tabla_parquet)
print("Cantidad de columnas: ",datos_tabla_parquet.num_columns)
print("Nombre columnas: ",datos_tabla_parquet.column_names)
print("Cantidad de registros: ",datos_tabla_parquet.num_rows)

Tipo de dato:  <class 'pyarrow.lib.Table'>
Tipos de datos:  pyarrow.Table
review_id: string
user_id: string
business_id: string
stars: double
useful: int64
funny: int64
cool: int64
text: string
date: string
----
review_id: [["KU_O5udG6zpxOg-VcAEodg","BiTunyQ73aT9WBnpR9DZGw","saUsX_uimxRlCVr67Z4Jig","AqPFMleE6RsU23_auESxiA","Sx8TMOWLNuJBWer-0pcmoA","JrIxlS1TzJ-iCu79ul40cQ","6AxgBCNX_PNTOxmbRSwcKQ","_ZeMknuYdlQcUqng_Im3yg","ZKvDG2sBvHVdF5oBNUOpAQ","pUycOfUwM8vqX7KjRRhUEA",...,"VX_qZ3uK7SIomq1t4BzFnQ","aFPIbN3aj5pN0RmlUDkmgw","Ki7ncLCgVWK9Ysukwk0zJg","lJqnUK9gASgdyx9USoKzfw","W8I3wWhb9hmglasrR6dePg","q4Z6NCR1IMRdpPorTD89Tg","KTg0i04SbVWGWHHJmGOI_A","ZuMNAPcArFtaGufe-nwGOA","Lohri9uZyvoNnH5z_NT6DA","9BaxxfWBmIsSPlx-kvRBDw"]]
user_id: [["mh_-eMZ6K5RLWhZyISBhwA","OyoGAe7OKpv6SyGZT5g77Q","8g_iMtfSiwikVnbP2etR0A","_7bHUi9Uuf5__HHc_Q8guQ","bcjbaE6dDog4jkNY91ncLQ","eUta8W_HdHMXPzLBBZhL1A","r3zeYsv1XFBRA4dJpL78cw","yfFzsLmaWF2d4Sr0UNbBgg","wSTuiTk-sKNdcFyprzZAjg","59MxRhNVhU9MYndMkz0wtw",...,"8TY

- - -

`Datasets: user`

Tipo de archivo Parquet

Procedimieto que permite separar el dataset user en 10 archivos tipo parquet con aproximadamente 150.000 registros cada archivo.<br>
El datasets user tiene aproximadamente: 2.105.597 registros.<br>

El siguiente procedimiento lee el datasets original Parquet, aproximadamente de 2.6Gb y lo separa en archivos tipo parquet.

In [ ]:
archivo_user_parquet = "../Datasets/Datasets_Yelp/user/user.parquet"             # Ruta y nombre del Archivo original
datos_user_parquet = pq.read_table(archivo_user_parquet)                        # Se lee y se carga el archivo como una tabla
cantidad_registros = 150000                                                     # Cantidad de registros por archivos separados
for ind in range(0, datos_user_parquet.num_rows, cantidad_registros):           # Un ciclo para grupo de registros
    info_cantidad_registros = datos_user_parquet.slice(ind, cantidad_registros) # Se extrae grupos de cantidad_registros registros
    nombre_archivo = "user_" + str(ind // cantidad_registros) + ".parquet"      # Se arma el nombre del archivo
    ruta_archivo = os.path.join('../Datasets/Datasets_Yelp/user/', nombre_archivo) # La ruta donde se guardara el archivo
    pq.write_table(info_cantidad_registros, ruta_archivo)                       # Se graba en el directorio

Se cargan los datasets que contienen grupos de registros ya separados. 

In [4]:
directorio = "../Datasets/Datasets_Yelp/user/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "user_*.parquet")
df_list = []                                    # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                      # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_user = pq.read_table(source=archivo)  # Se crea una tabla con cada archivo
    df = tabla_user.to_pandas()                 # Se transforma la tabla en un dataframe
    df_list.append(df)                          # Se adicionan a los dataframe
df_user_concatenado = pd.concat(df_list)        # Se concatenan todos los DataFrames en uno solo
df_user_concatenado = df_user_concatenado.reset_index(drop=True)    # Se resetea el indice a uno secuencial y consecutivo

Diccionario de datos del dataset de user.

| # | Column | Dtype | RealType |
| --------- | --------- | --------- | --------- |
| 0 | user_id | object | string |
| 1 | name| object | string | 
| 2 | review_count | int64 | int64 | 
| 3 | yelping_since | object | date YYYY-MM-DD |
| 4 | useful | int64 | int64 | 
| 5 | funny | int64 | int64 |  
| 6 | cool | int64 | int64 |
| 7 | elite | object | list int64 |
| 8 | friends | object | list string |
| 9 | fans | int64 | int64 |
| 10 | average_stars | float64 | float64 |
| 11 | compliment_hot | int64 | int64 |  
| 12 | compliment_more | int64 | int64 |  
| 13 | compliment_profile | int64 | int64 |  
| 14 | compliment_cute | int64 | int64 |  
| 15 | compliment_list | int64 | int64 |  
| 16 | compliment_note | int64 | int64 |  
| 17 | compliment_plain | int64 | int64 |  
| 18 | compliment_cool | int64 | int64 |  
| 19 | compliment_funny | int64 | int64 |  
| 20 | compliment_writer | int64 | int64 |  
| 21 | compliment_photos | int64 | int64 | 

Para su manipulacion se reduce el dataset a 5 años de registros y se ajustan los campos:<br> 
. yelping_since al formato: YYYY-MM-DD<br>

In [5]:
print('Datos iniciales desde: ', df_user_concatenado['yelping_since'].min(), 'hasta: ',df_user_concatenado['yelping_since'].max())
df_user_concatenado['yelping_since'] = pd.to_datetime(df_user_concatenado['yelping_since'])
df_user_2018_to_2022 = df_user_concatenado[df_user_concatenado['yelping_since'].dt.year.between(2018, 2022)]
df_user_2018_to_2022 = df_user_2018_to_2022.reset_index(drop=True)
df_user_2018_to_2022['yelping_since'] = pd.to_datetime(df_user_2018_to_2022['yelping_since']).dt.strftime('%Y-%m-%d')
print('Datos finales desde: ', df_user_2018_to_2022['yelping_since'].min(), 'hasta: ',df_user_2018_to_2022['yelping_since'].max())

Datos iniciales desde:  2004-10-12 08:46:11 hasta:  2022-01-19 17:15:47
Datos finales desde:  2018-01-01 hasta:  2022-01-19


Se crea un archivo en el directorio user con el dataset reducido en cantidad de registros para 5 años

In [6]:
table = pa.Table.from_pandas(df_user_2018_to_2022)                              # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
nombre_archivo = "user-reducido.parquet"                                        # Se arma el nombre del archivo
ruta_archivo = os.path.join('../Datasets/Datasets_Yelp/user/', nombre_archivo)  # La ruta donde se guardara el archivo
pq.write_table(table, ruta_archivo)                                             # Se graba en el directorio

Debido a limitaciones del repositorio de GitHob, donde solo permite archivos de tamaño maximo de 100MB, se procede a generar archivos que no excedan dicho tamaño.<br>
. Cantidad total de registros:  329174<br>
. Cantidad de registros aproximado por archivos: 50000<br>
. Cantidad de archivos: 7

Procedimiento para crear archivos separados de grupos de registros donde el tamaño de cada archivo no exceda de 50Mb

In [23]:
archivo_user_parquet = "../Datasets/Datasets_Yelp/user/user-reducido.parquet"   # Ruta y nombre del Archivo original
datos_user_parquet = pq.read_table(archivo_user_parquet)                        # Se lee y se carga el archivo como una tabla
cantidad_registros = 50000                                                      # Cantidad de registros por archivos separados
for ind in range(0, datos_user_parquet.num_rows, cantidad_registros):           # Un ciclo para grupo de registros
    info_cantidad_registros = datos_user_parquet.slice(ind, cantidad_registros) # Se extrae grupos de cantidad_registros registros
    nombre_archivo = "user-reducido_" + str(ind // cantidad_registros) + ".parquet"  # Se arma el nombre del archivo
    ruta_archivo = os.path.join('../Datasets/Datasets_Yelp/user/', nombre_archivo) # La ruta donde se guardara el archivo
    pq.write_table(info_cantidad_registros, ruta_archivo)                       # Se graba en el directorio

Cargando los archivos parquet reducidos como un Dataframe

In [25]:
directorio = "../Datasets/Datasets_Yelp/user/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "user-reducido_*.parquet")
df_list = []                                    # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                      # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_user = pq.read_table(source=archivo)  # Se crea una tabla con cada archivo
    df = tabla_user.to_pandas()                 # Se transforma la tabla en un dataframe
    df_list.append(df)                          # Se adicionan a los dataframe
df_user_reducido_concatenado = pd.concat(df_list)                                   # Se concatenan todos los DataFrames en uno solo
df_user_reducido_concatenado = df_user_reducido_concatenado.reset_index(drop=True)  # Se resetea el indice a uno secuencial y consecutivo

In [26]:
df_user_reducido_concatenado.head(4)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,RDlBwvQe5oEQ8STtj6a5rg,Dee,4,2018-11-06,5,0,2,,"FrLfQoTXb7k68ZqBQTHSew, 88wN6ysc8Z1LSgaa-Au2MA...",0,...,0,0,0,0,0,0,0,0,0,0
1,wZ5mQSbZR5UpGpnddGT7mw,Odalis,178,2018-04-15,542,95,429,"2018,2019,20,20,2021","zGAGITTaxhi2J14604F_iA, WoMHyzfYFI8juti3DbaIvg...",22,...,3,0,0,0,21,29,33,33,21,26
2,Oj8Ohm_2OlV6hjKMsQn69w,Emily,2,2018-07-01,1,0,0,,"YQhTRxurCw9uNe3stFyHHA, XUbfn0nkZgcsLixJJ_2WKQ...",0,...,0,0,0,0,0,0,0,0,0,0
3,LRjvkbcZ5S4efH0xAVfkxw,Maggie,1,2018-09-08,2,0,0,,"VNHx2MZANbDBZml5eNMP2g, 5CmrzlkGIglc5A-4b4S8Ig...",0,...,0,0,0,0,0,0,0,0,0,0


`Datasets: business`

Tipo de archivo csv

Se carga el archivo .csv, se elimina la columna Unnamed y se resetea el index

In [139]:
ruta_archivo_cardano = "../Datasets/Datasets_Yelp/business/df_business_yelp.csv"
df_business_yelp = pd.read_csv(ruta_archivo_cardano)
# Se elimina la columna Unnamed del dataframe
df_business_yelp = df_business_yelp.loc[:, ~df_business_yelp.columns.str.contains('^Unnamed')]
# Se eliminan las columnas repetidas que contienen .1 en su nombre
df_business_yelp = df_business_yelp.loc[:, ~df_business_yelp.columns.str.contains('.1')]
# Se eliminan los registros duplicadas
df_business_yelp.drop_duplicates(inplace=True)
# Se eliminan los registros que contienen null en el campo state
df_business_yelp = df_business_yelp[df_business_yelp['state'].notna()]
df_business_yelp = df_business_yelp[df_business_yelp['address'].notna()]
# Se resetea el index
df_business_yelp = df_business_yelp.reset_index(drop=True)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (15,16,17,18,19,20,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Contar los registros que tienen Null

In [140]:
df_business_yelp.isnull().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code        17
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      12873
categories        102
hours           22112
dtype: int64

Se eliminan los estados que no seran tomados en cuenta

In [137]:
lista_estados = ['Tennessee', 'Ohio', 'North Carolina', 'Georgia', 'Texas',
                 'Michigan', 'Alabama', 'Indiana', 'Arizona', 'Florida']
df_list = []                 
for estado in lista_estados:                 
    df = df_business_yelp[df_business_yelp.address.str.contains(estado)]   
    df_list.append(df)  
df_business_yelp_reducido = pd.concat(df_list)   
df_business_yelp_reducido = df_business_yelp_reducido.reset_index(drop=True)  # Se resetea el indice a uno secuencial y consecutivo

In [138]:
for estado in lista_estados: 
    print(estado, "Esta en: ", df_business_yelp.address.str.contains(estado).sum(), " direcciones")

Tennessee Esta en:  6  direcciones
Ohio Esta en:  44  direcciones
North Carolina Esta en:  0  direcciones
Georgia Esta en:  20  direcciones
Texas Esta en:  2  direcciones
Michigan Esta en:  283  direcciones
Alabama Esta en:  33  direcciones
Indiana Esta en:  75  direcciones
Arizona Esta en:  33  direcciones
Florida Esta en:  379  direcciones


In [135]:
for estado in lista_estados: 
    print(estado, "Esta en: ", df_business_yelp_reducido.address.str.contains(estado).sum(), " direcciones")

Tennessee Esta en:  6  direcciones
Ohio Esta en:  44  direcciones
North Carolina Esta en:  0  direcciones
Georgia Esta en:  20  direcciones
Texas Esta en:  2  direcciones
Michigan Esta en:  283  direcciones
Alabama Esta en:  33  direcciones
Indiana Esta en:  75  direcciones
Arizona Esta en:  33  direcciones
Florida Esta en:  379  direcciones


In [ ]:
directorio = "../Datasets/Datasets_Yelp/user/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "user-reducido_*.parquet")
df_list = []                                    # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                      # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_user = pq.read_table(source=archivo)  # Se crea una tabla con cada archivo
    df = tabla_user.to_pandas()                 # Se transforma la tabla en un dataframe
    df_list.append(df)                          # Se adicionan a los dataframe
df_user_reducido_concatenado = pd.concat(df_list)                                   # Se concatenan todos los DataFrames en uno solo
df_user_reducido_concatenado = df_user_reducido_concatenado.reset_index(drop=True)  # Se resetea el indice a uno secuencial y consecutivo

In [121]:
print(df_business_yelp.shape)
print(df_business_yelp_reducido.shape)

(145216, 14)
(875, 14)


In [118]:
print((df_business_yelp_reducido['address'] == 'Ohio'))

437       False
452       False
1124      False
1359      False
2346      False
          ...  
142831    False
142886    False
142911    False
143297    False
144058    False
Name: address, Length: 379, dtype: bool


Se salva el archivo en formato tipo parqet

In [141]:
df_business_yelp.to_parquet('../Datasets/Datasets_Yelp/business/business.parquet')

Se cargan los datasets que contienen grupos de registros ya separados. 

In [52]:
directorio = "../Datasets/Datasets_Yelp/business/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "business*.parquet")
df_list = []                                    # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                      # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_user = pq.read_table(source=archivo)  # Se crea una tabla con cada archivo
    df = tabla_user.to_pandas()                 # Se transforma la tabla en un dataframe
    df_list.append(df)                          # Se adicionan a los dataframe
df_business_concatenado = pd.concat(df_list)    # Se concatenan todos los DataFrames en uno solo
df_business_concatenado = df_business_concatenado.reset_index(drop=True)    # Se resetea el indice a uno secuencial y consecutivo

In [45]:
df_business_concatenado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145216 entries, 0 to 145215
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   145216 non-null  object 
 1   name          145216 non-null  object 
 2   address       145216 non-null  object 
 3   city          145216 non-null  object 
 4   state         145216 non-null  object 
 5   postal_code   145199 non-null  object 
 6   latitude      145216 non-null  float64
 7   longitude     145216 non-null  float64
 8   stars         145216 non-null  float64
 9   review_count  145216 non-null  int64  
 10  is_open       145216 non-null  int64  
 11  attributes    132343 non-null  object 
 12  categories    145114 non-null  object 
 13  hours         123104 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 15.5+ MB


In [46]:
df_business_concatenado.head(2).T

,0,1
business_id,MTSW4McQd7CbVtyjqoe9mw,mWMc6_wTdE0EUBKIGXDVfA
name,St Honore Pastries,Perkiomen Valley Brewery
address,935 Race St,101 Walnut St
city,Philadelphia,Green Lane
state,CA,MO
postal_code,19107,18054
latitude,39.9555,40.3382
longitude,-75.1556,-75.4717
stars,4,4.5
review_count,80,13


- - -
+ DataSet de Google Maps !!!
- - -

`Datasets: metadata-sitios`

Tipos de archivos JSon

El directorio metadata-sitios, esta compuesto por un grupo de archivos JSon, cada archivo contiene aproximadamente: 275.001 registros, por lo que se crea un procedimiento que permite crear de forma automatica un archivo parquet para cada uno de los archivos JSon en el directorio.

El siguiente procedimiento lee los datasets original JSon, aproximadamente de 870Mb y lo separa en archivos tipo parquet.

In [ ]:
directorio = "../Datsets/Datasets_Google_Maps/metadata-sitios/"                     # Ruta del directorio con los archivos JSon con la metadata-sitios
archivos_metadatos_sitios = os.listdir(directorio)                                  # Se guarda el listado de los archivos contenidos en el directorio

for archivo in archivos_metadatos_sitios:                                           # Se realiza un ciclo para cada archivo en el directorio
    archivo_metadata_sitios_json = directorio + archivo                             # Se le asigna la ruta y el nombre del archivo.
    with jsonlines.open(archivo_metadata_sitios_json) as reader:                    # Se obtiene el grupo de registros por cada archivo
        lista_datos = list(reader)                                                  # Lee todos los registros y crea una lista
        total_registros = len(lista_datos)                                          # Se tiene el total de registros, esto para separar en varios archivos.
        for ind in range(0, total_registros, total_registros):                      # Se crea un archivo parquet para cada grupo de registors
            iformacion_registros = lista_datos[ind:ind+total_registros]             # Se extrae grupos de cantidad_registros registros
            df = pd.DataFrame(iformacion_registros)                                 # Se crea un dataframe con el grupo de registros extraidos
            table = pa.Table.from_pandas(df)                                        # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
            nombre_archivo = "metadata-sitios_" + archivo[0: archivo.index('.')] + ".parquet"                   # Se arma el nombre del archivo
            ruta_archivo = os.path.join('../Datsets/Datasets_Google_Maps/metadata-sitios/', nombre_archivo)     # La ruta donde se guardara el archivo
            pq.write_table(table, ruta_archivo)                                     # Se crea el archivo con formato parquet en el directorio

Se cargan los datasets que contienen grupos de registros ya separados. 

In [3]:
directorio = "../Datasets/Datasets_Google_Maps/metadata-sitios/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "metadata-sitios_*.parquet")
df_list = []                                                          # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                                            # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_sitios = pq.read_table(source=archivo)                      # Se crea una tabla con cada archivo
    df = tabla_sitios.to_pandas()                                     # Se transforma la tabla en un dataframe
    df_list.append(df)                                                # Se adicionan a los dataframe
df_sitios_concatenado = pd.concat(df_list)                            # Se concatenan todos los DataFrames en uno solo
df_sitios_concatenado = df_sitios_concatenado.reset_index(drop=True)  # Se resetea el indice a uno secuencial y consecutivo

Diccionario de datos metada-sitios

| # | Column | Dtype | RealType |
| --------- | --------- | --------- | --------- |
| 0 | user_id | object | string |
| 0 | name | object | string |
| 1 | address | object | string |
| 2 | gmap_id | object | string |
| 3 | description | object | string |
| 4 | latitude | float64 | float64 |
| 5 | longitude | float64 | float64 |
| 6 | category | object | list string |
| 7 | avg_rating | float64 | float64 |
| 8 | num_of_reviews | int64 | float64 | 
| 9 | price | object | float64 $$|
| 10 | hours | object | list time|
| 11 | MISC | object | list string |
| 12 | state | object | string |
| 13 | relative_results | object | list | 
| 14 | url | object | string |

Se trabaja con una copia de los registros

In [6]:
print(df_sitios_concatenado.shape)
df_sitios_concatenado_copia = df_sitios_concatenado.copy()

(3025011, 15)


Se eliminan los registros que no tengan valor en el campo: address, description

In [7]:
df_sitios_concatenado_copia = df_sitios_concatenado_copia[df_sitios_concatenado_copia['address'].notna()]
print("Sin datos None en address: ",df_sitios_concatenado_copia.shape)

Sin datos None en address:  (2944500, 15)


In [44]:
df_sitios_concatenado_copia.head().T

,0,1,2,3,4
name,Porter Pharmacy,City Textile,San Soo Dang,Nova Fabrics,Nobel Textile Co
address,"Porter Pharmacy, 129 N Second St, Cochran, GA ...","City Textile, 3001 E Pico Blvd, Los Angeles, C...","San Soo Dang, 761 S Vermont Ave, Los Angeles, ...","Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...","Nobel Textile Co, 719 E 9th St, Los Angeles, C..."
gmap_id,0x88f16e41928ff687:0x883dad4fd048e8f8,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,0x80c2c778e3b73d33:0xbdc58662a4a97d49,0x80c2c89923b27a41:0x32041559418d447,0x80c2c632f933b073:0xc31785961fe826a6
description,None,None,None,None,None
latitude,32.3883,34.0189,34.0581,34.0237,34.0367
longitude,-83.3571,-118.215,-118.292,-118.233,-118.249
category,[Pharmacy],[Textile exporter],[Korean restaurant],[Fabric store],[Fabric store]
avg_rating,4.9,4.5,4.4,3.3,4.3
num_of_reviews,16,6,18,6,7
price,None,None,None,None,None


Para su manipulacion se reduce el dataset a solo los 10 estados a evaluar:<br>
['Tennessee', 'review-Ohio', 'North_Carolina', 'Georgia', 'Texas', 'Michigan', 'Alabama', 'Indiana', 'Arizona', 'Florida']

In [15]:
lista_estados = ['Tennessee', 'Ohio', 'North_Carolina', 'Georgia', 'Texas',
                 'Michigan', 'Alabama', 'Indiana', 'Arizona', 'Florida']

In [31]:
# lista_estados = ['Arizona', 'Florida']
lista_estados = ['Tennessee', 'Ohio', 'North Carolina', 'Georgia', 'Texas',
                 'Michigan', 'Alabama', 'Indiana', 'Arizona', 'Florida']
df_sitios_lista = [] 
for estado in lista_estados:
    df_sitios = df_sitios_concatenado_copia[df_sitios_concatenado_copia.address.str.contains(estado)]
    df_sitios_lista.append(df_sitios)
df_sitios_reducidos = pd.concat(df_sitios_lista)
df_sitios_reducidos = df_sitios_reducidos.reset_index(drop=True)

Se crea un archivo en el directorio metadata-sitios con el dataset reducido en cantidad de registros para 10 estados

In [34]:
table = pa.Table.from_pandas(df_sitios_reducidos)                                # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
nombre_archivo = "metadata-sitios_reducido.parquet"                              # Se arma el nombre del archivo
ruta_archivo = os.path.join('../Datasets/Datasets_Google_Maps/metadata-sitios/', nombre_archivo)  # La ruta donde se guardara el archivo
pq.write_table(table, ruta_archivo)                                              # Se graba en el directorio

Se cargan los datos del metadata-sitios reducidos

In [37]:
archivo = "../Datasets/Datasets_Google_Maps/metadata-sitios/metadata-sitios_reducido.parquet"
tabla_sitios = pq.read_table(source=archivo)                      # Se crea una tabla con cada archivo
df_sitios = tabla_sitios.to_pandas()                              # Se transforma la tabla en un dataframe

In [39]:
df_sitios.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Turkey Creek,"Turkey Creek, Tennessee",0x8864a58f0058d779:0x77f0f0f82ba4920e,None,35.998232,-87.389478,[River],3.8,5,None,None,None,None,None,https://www.google.com/maps/place/Tennessee/da...
1,Shelter Insurance - Nathan Mizer,"Shelter Insurance - Nathan Mizer, 107A W Tenne...",0x885c336f4bd36d55:0x480c0e91ddfd240d,None,36.026326,-84.243362,"[Insurance agency, Auto insurance agency, Home...",4.3,3,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x885c336fe87d6049:0x12141fa4a69f85f5, 0x885c...",https://www.google.com/maps/place//data=!4m2!3...
2,Flat Creek,"Flat Creek, Tennessee 37878",0x885ea23c1631d3df:0xeb621b8b97f25df6,None,35.673797,-83.894229,[River],5.0,4,None,None,None,None,None,https://www.google.com/maps/place/Tennessee+37...
3,Hurricane Branch,"Hurricane Branch, Tennessee 37882",0x885ea154ea071f21:0x1a1ec761c433c2a7,None,35.677006,-83.857697,[River],5.0,1,None,None,None,None,None,https://www.google.com/maps/place/Tennessee+37...
4,Autism Tennessee,"Autism Tennessee, 2607 Winford Ave, Nashville,...",0x886467cd4482dcff:0x636916ab972b869d,None,36.114866,-86.753344,[Social services organization],5.0,4,None,"[[Thursday, 10AM–3PM], [Friday, 10AM–3PM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 10AM,"[0x8864657265da0d5f:0x8b83c74c7fb3b1a6, 0x8864...",https://www.google.com/maps/place//data=!4m2!3...


- - -
`Datasets: metadata-estados`

Compuesto por un directorio para cada estado y dentro un grupo de archivos tipo JSon

El directorio metadata-estados, esta compuesto por un grupo de directoros, uno por cada estado, y un conjunto de archivos JSon, cada archivo contiene aproximadamente: 150.000 registros, por lo que se crea un procedimiento que permite crear de forma automatica un archivo parquet para cada uno de los archivos JSon en el directorio de cada estado.

In [ ]:
directorio_origen = "../Datsets/Datasets_Google_Maps/reviews-estados/"                  # Debido a la cantidad de directorios y archivos se permitira tener separados los archivos de origen.
directorio_destino = "../Datsets/Datasets_Google_Maps/reviews-estados/"                 # Debido a la cantidad de directorios y archivos se permitira tener separados los archivos de destido.
directorios_reviews_estados = os.listdir(directorio_origen)                             # Se obtiene un listado de todos los subdirectorios del directorio origen.
for sub_dir in directorios_reviews_estados:                                             # Ciclo por cada subdirectorio
    archivos_reviews_estados = os.listdir(directorio_origen + sub_dir)                  # Se guarda el directorio y el subdirectorio a utilizar
    os.mkdir(directorio_destino + sub_dir)                                              # Directorio donde se colocaran los archivos parquet
    for archivo in archivos_reviews_estados:                                            # Ciclo por cada archivo en cada subdirectorio
        archivo_review_estados_json = directorio_origen + sub_dir + '/' + archivo       # Se guarda el directorio origen, el subdirectorio y el nombre del archivo para ser leido.
        with jsonlines.open(archivo_review_estados_json) as reader:                     # Se lee el archivo JSon
            lista_datos = list(reader)                                                  # Lee todos los registros y crea una lista
            total_registros = len(lista_datos)                                          # Se tiene el total de registros, esto para separar en varios archivos.
            for ind in range(0, total_registros, total_registros):                      # Un ciclo para los registros de cada archivo JSon
                iformacion_registros = lista_datos[ind:ind+total_registros]             # Se extrae grupos de cantidad_registros registros
                df = pd.DataFrame(iformacion_registros)                                 # Se crea un dataframe con el grupo de registros extraidos
                table = pa.Table.from_pandas(df)                                        # Se crea una tabla tipo pyarrow del dataframe para guardarlo en archivos separados.
                nombre_archivo = sub_dir + "_" + archivo[0: archivo.index('.')] + ".parquet"   # Se arma el nombre del archivo
                ruta_archivo = os.path.join(directorio_destino + sub_dir, nombre_archivo)      # La ruta donde se guardara el archivo
                pq.write_table(table, ruta_archivo)                                     # Se crea el archivo parquet en el directorio destino

Cargando la informacion de solo un estado

In [43]:
nombre_archivo = 'Texas'
directorio = "../Datasets/Datasets_Google_Maps/reviews-estados/review-" + nombre_archivo + "/"
ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "review-*.parquet")
df_list = []                                                            # Un dataframe vacio para ir adicionando
for archivo in ls_archivo:                                              # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
    tabla_estados = pq.read_table(source=archivo)                       # Se crea una tabla con cada archivo
    df = tabla_estados.to_pandas()                                      # Se transforma la tabla en un dataframe
    df_list.append(df)                                                  # Se adicionan a los dataframe
df_estados_concatenado = pd.concat(df_list)                             # Se concatenan todos los DataFrames en uno solo
df_estados_concatenado['estado'] = nombre_archivo
df_estados_concatenado = df_estados_concatenado.reset_index(drop=True)  # Se resetea el indice a uno secuencial y consecutivo
archivo_estado_csv = directorio + 'review-' + nombre_archivo + ".csv"
df_estados_concatenado.to_csv(archivo_estado_csv, index=False)

In [51]:
archivo_estado_csv = "../Datasets/Datasets_Google_Maps/reviews-estados/review-North_Carolina/review-North_Carolina.csv"
df = pd.read_csv(archivo_estado_csv)
df.shape

(2250000, 9)

Cargando la informacion de todos los estados

In [7]:
directorio_origen = "../Datasets/Datasets_Google_Maps/reviews-estados/"    # Debido a la cantidad de directorios y archivos se permitira tener separados los archivos de origen.
directorios_reviews_estados = os.listdir(directorio_origen)                # Se obtiene un listado de todos los subdirectorios del directorio origen.
for sub_dir in directorios_reviews_estados:                                # Ciclo por cada subdirectorio
    nombre_archivo = sub_dir.replace("review-", '')
    archivos_reviews_estados = os.listdir(directorio_origen + sub_dir)     # Se guarda el directorio y el subdirectorio a utilizar
    directorio = directorio_origen + sub_dir + "/"
    ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "*.parquet")
    df_list = []                                                            # Un dataframe vacio para ir adicionando
    for archivo in ls_archivo:                                              # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
        # print(directorio, nombre_archivo)
        # print(ls_archivo)
        tabla_estados = pq.read_table(source=archivo)                       # Se crea una tabla con cada archivo
        df = tabla_estados.to_pandas()                                      # Se transforma la tabla en un dataframe
        # df['estado'] = nombre_archivo
        df_list.append(df)                                                  # Se adicionan a los dataframe
    df_estados_concatenado = pd.concat(df_list)                             # Se concatenan todos los DataFrames en uno solo
    df['estado'] = nombre_archivo
df_estados_concatenado = df_estados_concatenado.reset_index(drop=True)      # Se resetea el indice a uno secuencial y consecutivo        

In [45]:
directorio_origen = "../Datasets/Datasets_Google_Maps/reviews-estados/"    # Debido a la cantidad de directorios y archivos se permitira tener separados los archivos de origen.
directorios_reviews_estados = os.listdir(directorio_origen)                # Se obtiene un listado de todos los subdirectorios del directorio origen.
for sub_dir in directorios_reviews_estados:                                # Ciclo por cada subdirectorio
    nombre_archivo = sub_dir.replace("review-", '')
    archivos_reviews_estados = os.listdir(directorio_origen + sub_dir)     # Se guarda el directorio y el subdirectorio a utilizar
    directorio = directorio_origen + sub_dir + "/"
    ls_archivo = listar_archivos_segun_patron_con_directorio(directorio, "*.csv")
    df_list = []                                                            # Un dataframe vacio para ir adicionando
    for archivo in ls_archivo:                                              # Leer cada archivo parquet en un DataFrame de Pandas y adicionarlo
        df = pd.read_csv(archivo)
        df_list.append(df)                                                  # Se adicionan a los dataframe
    df_estados_concatenado = pd.concat(df_list)                             # Se concatenan todos los DataFrames en uno solo
df_estados_concatenado = df_estados_concatenado.reset_index(drop=True)      # Se resetea el indice a uno secuencial y consecutivo        

In [50]:
df_estados_concatenado.shape

(2850000, 9)

In [49]:
df_estados_concatenado.tail(20).T

,2849980,2849981,2849982,2849983,2849984,2849985,2849986,2849987,2849988,2849989,2849990,2849991,2849992,2849993,2849994,2849995,2849996,2849997,2849998,2849999
user_id,116027333961946450036,101682319947243887670,114076018014845789116,116410452070326430922,113346740550680886125,112172420842429498318,103423168954627189806,115332274602197044834,104445444863675602185,112970752410215005542,111541954676733314084,101455115901962250159,117050501385090268935,116209957654170876138,104864780221475782836,113432710667574630477,114437354870659097984,110393435648440523598,103517617368795862805,103232669323935534602
name,Google Me,Chandrally Mahajan,Cynthia Hild,Chun Chun,Mary Whitley,raymond cribbs,Bonnie Martin,Leslie Nugent,Jackie Strickland,Greg Wainwright,FlipTheSwitch (FlipTheSwitch),Susan Dugger,Steve Brooks,sherry malovich,Gary Peterson,Ricky Rice,John Davis,Georgia Mcfadden,Freddie Davis,Autumn Bozarth
time,1412308477217,1574728423657,1588809149105,1574728154852,1616522076274,1607454563101,1617391403010,1579124844264,1584470569359,1578174179270,1530637403425,1499618592061,1520484245437,1581576060516,1561087785711,1544911047205,1543609738956,1538099584340,1520106492564,1625153702009
rating,5,5,5,5,5,5,5,5,4,4,5,5,5,4,4,4,5,5,5,4
text,NaN,NaN,NaN,NaN,They are miracle workers! I let my sister colo...,Tanning. 5-Stars! Extremely helpful and unders...,Hydro bed is the @#%$,Great place for hair 💇 nails 💅 and tanning! Gr...,LuAnn listens to what you want and then does i...,The stylist are very nice and did an amazing j...,"Always great, good prices and offers nails enh...",Love Luanne she always does an awesome job on ...,"Quick, friendly service. Very homey.",Nails look great.,Wish she stop having to move,Very professional.,Great service,Give great hair cuts and styling,Great place / Good work,They definitely know what they are doing. My d...
pics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[{'url': array(['https://lh5.googleusercontent...
resp,"{'text': 'Thank you for 5 Stars review.', 'tim...",{'text': 'Thanks for 5 Star Review. We are her...,"{'text': ""We treat all of our patient's as Sta...",{'text': 'Sincere Thanks and really appreciate...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'text': 'Thank you! My pleasure', 'time': 154...",NaN,NaN,NaN,{'text': 'I am sorry if you feel you received ...
gmap_id,0x88d9aaf5e68c3d11:0x6dded03ffc9a0caa,0x88d9aaf5e68c3d11:0x6dded03ffc9a0caa,0x88d9aaf5e68c3d11:0x6dded03ffc9a0caa,0x88d9aaf5e68c3d11:0x6dded03ffc9a0caa,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d,0x88e61aa5df038465:0x580976b6e9c5313d
estado,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida,Florida


- - -